In [3]:
 # Crawling에 필요한 module 선언
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# (201105) 추가한 모듈
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

# pandas
import pandas as pd

In [2]:
def get_youtube_video_one_data(url):
    # webdriver 초기화
    # 아래 3줄은 selenium으로 크롤링을 할 때 꼭 기본적으로 해주어야 하는 세팅
    driver_path ='./chromedriver'
    driver = webdriver.Chrome(driver_path)
    driver.implicitly_wait(5) # or bigger second
    
    # 열고자 하는 채널 -> 동영상 목록으로 된 url 페이지를 엶
    driver.get(url)
    
    # 처음에는 1개 내지 2개정도만 제대로 데이터를 긁어 오는지 확인
    thumbnail_list = [] # 썸네일을 받을 수 있는 주소 저장용 리스트
    title_list = [] # 썸네일 제목을 저장하는 리스트
    
    try:
    # xpath의 경로는 '절대경로'로 정해줌
        thumbnail_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-rich-grid-renderer/div[5]/ytd-rich-item-renderer[1]/div/ytd-rich-grid-media/div[1]/ytd-thumbnail/a/yt-img-shadow/img'
        title_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-rich-grid-renderer/div[5]/ytd-rich-item-renderer[1]/div/ytd-rich-grid-media/div[1]/div/div[1]/h3/a/yt-formatted-string'
 
        # 이미지, 채널 프로필, 오버레이 타임이 곧바로 로드 되지 않을 때, 20초간 강제로 기다림
        thumb = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.XPATH, thumbnail_xpath)))

        # thumbnail 객체가 로드가 되었는지 확인
        if thumb is None:
            print("img is not loaded.")
        
        # 동영상 목록을 추가 불러오기 위해 스크롤 내림(pass) -> 다중 
        
        # 썸네일 주소를 리스트에 저장
        thumbnail = driver.find_element_by_xpath(thumbnail_xpath)
        thumbnail_url = thumbnail.get_attribute('src')
        thumbnail_list.append(thumbnail_url)
        
        # 타이틀을 리스트에 저장
        title = driver.find_element_by_xpath(title_xpath)
        title_list.append(title.text)
        
        # 리스트에 들어간 데이터 중간 확인
        print('title_text:', title.text, ' thumbnail_url: ', thumbnail_url)
        
    except Exception as e:
        print()
        print('Exception: ', e)
    
    # image와 title을 서로 같은 양을 모두 크롤링을 마쳤다면 driver를 닫아준다
    # assert len(thumbnail_list) == len(title_list)
    driver.close()
    return thumbnail_list, title_list

In [3]:
test_url = 'https://www.youtube.com/'
get_youtube_video_one_data(test_url)

title_text: [Playlist] 귀르가즘 제대로 주는 갬성 팝송 노래 모음 #2  thumbnail_url:  https://i.ytimg.com/vi/mxJGDa7ThbE/hq720.jpg?sqp=-oaymwEZCNAFEJQDSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLD2kF6j-nY8rIHfCDZqEIcUqGnuhQ


(['https://i.ytimg.com/vi/mxJGDa7ThbE/hq720.jpg?sqp=-oaymwEZCNAFEJQDSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLD2kF6j-nY8rIHfCDZqEIcUqGnuhQ'],
 ['[Playlist] 귀르가즘 제대로 주는 갬성 팝송 노래 모음 #2'])

In [4]:
# 유튜브 이미지와 타이틀을 크롤링 하는 함수 정의
# parameter : url
def get_youtube_video_one_data(url):
    # webdriver 초기화
    driver_path ='./chromedriver'
    driver = webdriver.Chrome(driver_path)
    driver.implicitly_wait(5) # or bigger second
    
    # 열고자 하는 채널 -> 동영상 목록으로 된 url 페이지를 엶
    driver.get(url)
    
    thumbnail_list = [] # 썸네일을 받을 수 있는 주소 저장용 리스트
    title_list = [] # 썸네일 제목을 저장하는 리스트
    overlay_time_list = [] # 동영상 재생시간들을 저장하는 리스트
    channel_profile_list = [] # 채널 프로필사진 주소 저장용 리스트
    channel_name_list = [] # 채널 이름 저장용 리스트
    hits_str_list = [] # 조회수 문자열 저장용 리스트
    update_date_list = [] # 업로드 시간 저장용 리스트
    
    # index = 1
    # 무한히 반복
    # while True:
    try:
        thumbnail_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-rich-grid-renderer/div[5]/ytd-rich-item-renderer[1]/div/ytd-rich-grid-media/div[1]/ytd-thumbnail/a/yt-img-shadow/img'
        title_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-rich-grid-renderer/div[5]/ytd-rich-item-renderer[1]/div/ytd-rich-grid-media/div[1]/div/div[1]/h3/a/yt-formatted-string'
        overlay_time_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-rich-grid-renderer/div[5]/ytd-rich-item-renderer[1]/div/ytd-rich-grid-media/div[1]/ytd-thumbnail/a/div[1]/ytd-thumbnail-overlay-time-status-renderer/span'
        channel_profile_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-rich-grid-renderer/div[5]/ytd-rich-item-renderer[1]/div/ytd-rich-grid-media/div[1]/div/a/yt-img-shadow/img'
        channel_name_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse[1]/ytd-two-column-browse-results-renderer/div[1]/ytd-rich-grid-renderer/div[5]/ytd-rich-item-renderer[1]/div/ytd-rich-grid-media/div[1]/div/div[1]/ytd-video-meta-block/div[1]/div[1]/ytd-channel-name/div/div/yt-formatted-string/a'
        hits_str_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse[1]/ytd-two-column-browse-results-renderer/div[1]/ytd-rich-grid-renderer/div[5]/ytd-rich-item-renderer[1]/div/ytd-rich-grid-media/div[1]/div/div[1]/ytd-video-meta-block/div[1]/div[2]/span[1]'
        update_date_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse[1]/ytd-two-column-browse-results-renderer/div[1]/ytd-rich-grid-renderer/div[5]/ytd-rich-item-renderer[1]/div/ytd-rich-grid-media/div[1]/div/div[1]/ytd-video-meta-block/div[1]/div[2]/span[2]'
        
        # 이미지, 채널 프로필, 오버레이 타임이 곧바로 로드 되지 않을 때, 20초간 강제로 기다림
        thumb = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.XPATH, thumbnail_xpath)))
        overlay = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.XPATH, overlay_time_xpath)))
        ch_profile = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.XPATH, channel_profile_xpath)))

        if (thumb or overlay or ch_profile) is None:
            print("img or overlay or ch_profile is not loaded.")
        
        # 동영상 목록을 추가 불러오기 위해 스크롤 내림(pass)
        
        # 썸네일 주소를 리스트에 저장
        thumbnail = driver.find_element_by_xpath(thumbnail_xpath)
        thumbnail_url = thumbnail.get_attribute('src')
        thumbnail_list.append(thumbnail_url)
        
        # 타이틀을 리스트에 저장
        title = driver.find_element_by_xpath(title_xpath)
        title_list.append(title.text)
        
        # 동영상 재생시간을 리스트에 저장
        overlay_time = driver.find_element_by_xpath(overlay_time_xpath)
        overlay_time_list.append(overlay_time.text)
        
        # 채널 프로필 사진 링크를 리스트에 저장
        channel_profile = driver.find_element_by_xpath(channel_profile_xpath)
        channel_profile_url = channel_profile.get_attribute('src')
        channel_profile_list.append(channel_profile_url)
        
        # 채널이름을 리스트에 저장
        channel_name = driver.find_element_by_xpath(channel_name_xpath)
        channel_name_list.append(channel_name.text)
        
        # 조회수 문자열을 리스트에 저장
        hits_str = driver.find_element_by_xpath(hits_str_xpath)
        hits_str_list.append(hits_str.text)
        
        # 업데이트 시간을 리스트에 저장
        update_date = driver.find_element_by_xpath(update_date_xpath)
        update_date_list.append(update_date.text)
        
        # 리스트에 들어간 데이터 중간 확인
#         print('title_text:', title.text, ' img_url: ', thumbnail_url)
#         print('overlay_time: ', overlay_time.text, ' channel_profile: ', channel_profile_url)
#         print('channel_name: ', channel_name.text, ' hits_str: ', hits_str.text, ' update_date: ', update_date.text) 
        
    except Exception as e:
        print()
        print('Exception: ', e)
    
    # image와 title을 서로 같은 양을 모두 크롤링을 마쳤다면 driver를 닫아준다
    # assert len(thumbnail_list) == len(title_list)
    driver.close()
    
    # pandas DataFrame 형태로 데이터를 테이블에 담아 리턴
    for thumbnail, title, overlay, channel_profile, channel_name, hits_str, update_date in zip(thumbnail_list, title_list, overlay_time_list, channel_profile_list, channel_name_list,
        hits_str_list, update_date_list):
        print(thumbnail, title, overlay, channel_profile, channel_name, hits_str, update_date)
    
    # making dataframe
    dic = {'thumbnail': thumbnail_list, 'title': title_list, 'overlay_time': overlay_time_list, 'channel_profile': channel_profile_list,
          'channel_name': channel_name_list, 'hits_str': hits_str_list, 'update_date': update_date_list}
    
    df = pd.DataFrame(dic)
    
    return df

In [5]:
# test code

# webdriver 초기화
# driver_path ='./chromedriver'
# driver = webdriver.Chrome(driver_path)
# driver.implicitly_wait(5) # or bigger second

# 열고자 하는 채널 -> 동영상 목록으로 된 url 페이지를 엶
# driver.get('https://www.youtube.com/')

# channel_name_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse[1]/ytd-two-column-browse-results-renderer/div[1]/ytd-rich-grid-renderer/div[5]/ytd-rich-item-renderer[1]/div/ytd-rich-grid-media/div[1]/div/div[1]/ytd-video-meta-block/div[1]/div[1]/ytd-channel-name/div/div/yt-formatted-string/a'
# hits_str_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse[1]/ytd-two-column-browse-results-renderer/div[1]/ytd-rich-grid-renderer/div[5]/ytd-rich-item-renderer[1]/div/ytd-rich-grid-media/div[1]/div/div[1]/ytd-video-meta-block/div[1]/div[2]/span[1]'
# update_date_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse[1]/ytd-two-column-browse-results-renderer/div[1]/ytd-rich-grid-renderer/div[5]/ytd-rich-item-renderer[1]/div/ytd-rich-grid-media/div[1]/div/div[1]/ytd-video-meta-block/div[1]/div[2]/span[2]'

# 채널이름을 리스트에 저장
# channel_name = driver.find_element_by_xpath(channel_name_xpath)
# channel_name_list.append(channel_name.text)

# 조회수 문자열을 리스트에 저장
# hits_str = driver.find_element_by_xpath(hits_str_xpath)
# hits_str_list.append(hits_str.text)

# 업데이트 시간을 리스트에 저장
# update_date = driver.find_element_by_xpath(update_date_xpath)
# update_date_list.append(update_date.text)

# print('channel_name: ', channel_name.text, ' hits_str: ', hits_str.text, ' update_date: ', update_date.text)

# driver.close()

In [6]:
# img compare
# '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-rich-grid-renderer/div[5]/ytd-rich-item-renderer[1]/div/ytd-rich-grid-media/div[1]/ytd-thumbnail/a/yt-img-shadow/img'
# '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer/div[1]/ytd-grid-video-renderer['+str(idx)+']/div[1]/ytd-thumbnail/a/yt-img-shadow/img'

In [7]:
# title compare
# '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-rich-grid-renderer/div[5]/ytd-rich-item-renderer[1]/div/ytd-rich-grid-media/div[1]/div/div[1]/h3/a/yt-formatted-string'
# '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer/div[1]/ytd-grid-video-renderer['+str(idx)+']/div[1]/div[1]/div[1]/h3/a'

In [8]:
test_url = 'https://www.youtube.com/'
get_youtube_video_one_data(test_url)

https://i.ytimg.com/vi/6GZstlm9EnQ/hq720.jpg?sqp=-oaymwEZCNAFEJQDSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLCo2IuqQWBI2GzRPx2-8SaO9nEa8w 🔥[No.1 채널 멜론둥이의 멜론차트]🔥 2020년 10월 31일 5주차🔥KPOP Chart🔥 최신 인기가요 노래 모음!!🔥🔥 4:18:25 https://yt3.ggpht.com/a-/AOh14GgpWtxti1qwWul5tiwKsokaNQF-zLeVshujwQ=s68-c-k-c0x00ffffff-no-rj-mo 멜론둥이 조회수 27만회 6일 전


,thumbnail,title,overlay_time,channel_profile,channel_name,hits_str,update_date
0,https://i.ytimg.com/vi/6GZstlm9EnQ/hq720.jpg?s...,🔥[No.1 채널 멜론둥이의 멜론차트]🔥 2020년 10월 31일 5주차🔥KPOP ...,4:18:25,https://yt3.ggpht.com/a-/AOh14GgpWtxti1qwWul5t...,멜론둥이,조회수 27만회,6일 전


In [4]:
# 유튜브 이미지와 타이틀을 크롤링 하는 함수 정의
# parameter : url
def get_youtube_main_video_all_data(url):
    # webdriver 초기화
    driver_path ='./chromedriver'
    driver = webdriver.Chrome(driver_path)
    driver.implicitly_wait(30) # or bigger second
    
    # 열고자 하는 채널 -> 동영상 목록으로 된 url 페이지를 엶
    driver.get(url)
    
    thumbnail_list = [] # 썸네일을 받을 수 있는 주소 저장용 리스트
    title_list = [] # 썸네일 제목을 저장하는 리스트
    overlay_time_list = [] # 동영상 재생시간들을 저장하는 리스트
    channel_profile_list = [] # 채널 프로필사진 주소 저장용 리스트
    channel_name_list = [] # 채널 이름 저장용 리스트
    hits_str_list = [] # 조회수 문자열 저장용 리스트
    update_date_list = [] # 업로드 시간 저장용 리스트
    
    # 첫 동영상 시작 시 시작번호(index = 1)
    index = 1
    # 한 번 스크롤하고 멈춤 시간 설정
    SCROLL_PAUSE_TIME = 2
    # last_height 초깃값 설정
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    # 무한히 반복, 유튜브 페이지는 무한 스크롤이기 때문에
    while True:
        try:
            thumbnail_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-rich-grid-renderer/div[5]/ytd-rich-item-renderer['+ str(index) +']/div/ytd-rich-grid-media/div[1]/ytd-thumbnail/a/yt-img-shadow/img'
            title_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-rich-grid-renderer/div[5]/ytd-rich-item-renderer['+ str(index) +']/div/ytd-rich-grid-media/div[1]/div/div[1]/h3/a/yt-formatted-string'
            overlay_time_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-rich-grid-renderer/div[5]/ytd-rich-item-renderer['+ str(index) +']/div/ytd-rich-grid-media/div[1]/ytd-thumbnail/a/div[1]/ytd-thumbnail-overlay-time-status-renderer/span'
            channel_profile_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-rich-grid-renderer/div[5]/ytd-rich-item-renderer['+ str(index) +']/div/ytd-rich-grid-media/div[1]/div/a/yt-img-shadow/img'
            channel_name_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse[1]/ytd-two-column-browse-results-renderer/div[1]/ytd-rich-grid-renderer/div[5]/ytd-rich-item-renderer['+ str(index) +']/div/ytd-rich-grid-media/div[1]/div/div[1]/ytd-video-meta-block/div[1]/div[1]/ytd-channel-name/div/div/yt-formatted-string/a'
            hits_str_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse[1]/ytd-two-column-browse-results-renderer/div[1]/ytd-rich-grid-renderer/div[5]/ytd-rich-item-renderer['+ str(index) +']/div/ytd-rich-grid-media/div[1]/div/div[1]/ytd-video-meta-block/div[1]/div[2]/span[1]'
            update_date_xpath = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse[1]/ytd-two-column-browse-results-renderer/div[1]/ytd-rich-grid-renderer/div[5]/ytd-rich-item-renderer['+ str(index) +']/div/ytd-rich-grid-media/div[1]/div/div[1]/ytd-video-meta-block/div[1]/div[2]/span[2]'

            # 이미지, 채널 프로필, 오버레이 타임이 곧바로 로드 되지 않을 때, 20초간 강제로 기다림
            thumb = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.XPATH, thumbnail_xpath)))
            overlay = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.XPATH, overlay_time_xpath)))
            ch_profile = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.XPATH, channel_profile_xpath)))

            if (thumb or overlay or ch_profile) is None:
                print("img or overlay or ch_profile is not loaded.")
            
#             if index == 5:
#                 break
            
            # 동영상 목록을 추가 불러오기 위해 스크롤 내림(pass)
            # index % 8 일 경우 스크롤을 내림(8개씩 다 볼 경우)
            # 스크롤의 높이를 직접 계산해서 하는 방식
            if index % 8 == 0:
                # 화면 최하단으로 스크롤 다운
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                
                # 페이지 로드를 기다림
                time.sleep(SCROLL_PAUSE_TIME)
                
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);")
                
                time.sleep(SCROLL_PAUSE_TIME)
                
                # 이동 후의 새로운 높이를 계산
                new_height = driver.execute_script("return document.body.scrollHeight")
                
                # 새로운 높이가 이전 높이와 변하지 않았을 경우(스크롤 다 내려서 갱신할게 없을 경우)
                if new_height == last_height:
                    break
                
                # 스크롤 다운이 된다면 스크롤 다운이 된 후의 창 높이를 새로운 높이로 갱신
                last_height = new_height
                

            # 썸네일 주소를 리스트에 저장
            thumbnail = driver.find_element_by_xpath(thumbnail_xpath)
            thumbnail_url = thumbnail.get_attribute('src')
            thumbnail_list.append(thumbnail_url)

            # 타이틀을 리스트에 저장
            title = driver.find_element_by_xpath(title_xpath)
            title_list.append(title.text)

            # 동영상 재생시간을 리스트에 저장
            overlay_time = driver.find_element_by_xpath(overlay_time_xpath)
            overlay_time_list.append(overlay_time.text)

            # 채널 프로필 사진 링크를 리스트에 저장
            channel_profile = driver.find_element_by_xpath(channel_profile_xpath)
            channel_profile_url = channel_profile.get_attribute('src')
            channel_profile_list.append(channel_profile_url)

            # 채널이름을 리스트에 저장
            channel_name = driver.find_element_by_xpath(channel_name_xpath)
            channel_name_list.append(channel_name.text)

            # 조회수 문자열을 리스트에 저장
            hits_str = driver.find_element_by_xpath(hits_str_xpath)
            hits_str_list.append(hits_str.text)

            # 업데이트 시간을 리스트에 저장
            update_date = driver.find_element_by_xpath(update_date_xpath)
            update_date_list.append(update_date.text)

            # 리스트에 들어간 데이터 중간 확인
    #         print('title_text:', title.text, ' img_url: ', thumbnail_url)
    #         print('overlay_time: ', overlay_time.text, ' channel_profile: ', channel_profile_url)
    #         print('channel_name: ', channel_name.text, ' hits_str: ', hits_str.text, ' update_date: ', update_date.text) 
            index += 1 # 다음 영상데이터로 넘어간다
        
        except AttributeError as e:
            print()
            print('Exception: ', e) 
    
    # image와 title을 서로 같은 양을 모두 크롤링을 마쳤다면 driver를 닫아준다
    # assert len(thumbnail_list) == len(title_list)
    driver.close()
    
    # pandas DataFrame 형태로 데이터를 테이블에 담아 리턴
    for thumbnail, title, overlay, channel_profile, channel_name, hits_str, update_date in zip(thumbnail_list, title_list, overlay_time_list, channel_profile_list, channel_name_list,
        hits_str_list, update_date_list):
        print(thumbnail, title, overlay, channel_profile, channel_name, hits_str, update_date)
    
    # making dataframe
    dic = {'thumbnail': thumbnail_list, 'title': title_list, 'overlay_time': overlay_time_list, 'channel_profile': channel_profile_list,
          'channel_name': channel_name_list, 'hits_str': hits_str_list, 'update_date': update_date_list}
    
    df = pd.DataFrame(dic)
    
    return df

In [6]:
test_url = 'https://www.youtube.com/'
get_youtube_main_video_all_data(test_url)

https://i.ytimg.com/vi/EUW6MxGg64A/hq720.jpg?sqp=-oaymwEZCNAFEJQDSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLAAbJjjUZe0QAmdpdP8vk_7wDlk9Q 🔥[No.1 채널 멜론둥이의 멜론차트]🔥 2020년 11월 10일 2주차🔥KPOP Chart🔥 최신 인기가요 노래 모음!!🔥🔥 5:51:31 https://yt3.ggpht.com/a-/AOh14GgpWtxti1qwWul5tiwKsokaNQF-zLeVshujwQ=s68-c-k-c0x00ffffff-no-rj-mo 멜론둥이 조회수 3만회 8시간 전
https://i.ytimg.com/vi/jzYklsjUCLE/hq720.jpg?sqp=-oaymwEZCNAFEJQDSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLCOA5fGJ_dv6ezf4rjZzYgRS_bQVQ 외국에서 아무것도 모르고 음식 주문하면 생기는 일.mp4 | 신서유기5 tvnbros5 EP.4 3:20 https://yt3.ggpht.com/a-/AOh14GgZvEHbDpHaZDO0dk9-XZKjSrSpecCT2f6Bmg=s68-c-k-c0x00ffffff-no-rj-mo tvN 조회수 129만회 2년 전
https://i.ytimg.com/vi/CtChAxrAttM/hq720.jpg?sqp=-oaymwEZCNAFEJQDSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLDEDkE_c3ktVENPY0-vKeG1ROWCIw 방에서 내보냈다고 흐느끼는 강아지.. 3:43 https://yt3.ggpht.com/a-/AOh14Ggu-FVZ7PjjsU6Ha7_hPtFeUscDaYqTmVALhw=s68-c-k-c0x00ffffff-no-rj-mo 아리둥절 Ari the Corgi 조회수 51만회 3주 전
https://i.ytimg.com/vi/RMIBC4F_GVY/hq720.jpg?sqp=-oaymwEZCNAFEJQDSFXyq4qpAwsIA

,thumbnail,title,overlay_time,channel_profile,channel_name,hits_str,update_date
0,https://i.ytimg.com/vi/EUW6MxGg64A/hq720.jpg?s...,🔥[No.1 채널 멜론둥이의 멜론차트]🔥 2020년 11월 10일 2주차🔥KPOP ...,5:51:31,https://yt3.ggpht.com/a-/AOh14GgpWtxti1qwWul5t...,멜론둥이,조회수 3만회,8시간 전
1,https://i.ytimg.com/vi/jzYklsjUCLE/hq720.jpg?s...,외국에서 아무것도 모르고 음식 주문하면 생기는 일.mp4 | 신서유기5 tvnbro...,3:20,https://yt3.ggpht.com/a-/AOh14GgZvEHbDpHaZDO0d...,tvN,조회수 129만회,2년 전
2,https://i.ytimg.com/vi/CtChAxrAttM/hq720.jpg?s...,방에서 내보냈다고 흐느끼는 강아지..,3:43,https://yt3.ggpht.com/a-/AOh14Ggu-FVZ7PjjsU6Ha...,아리둥절 Ari the Corgi,조회수 51만회,3주 전
3,https://i.ytimg.com/vi/RMIBC4F_GVY/hq720.jpg?s...,[라디오스타] 다현의 진기명기 유연성 개인기! 😨 20201028,2:36,https://yt3.ggpht.com/a-/AOh14Gg0jft0lRv6zvU8p...,MBCentertainment,조회수 287만회,1주 전
4,https://i.ytimg.com/vi/Fyd1BiP6ipg/hq720.jpg?s...,"[어디선가 들어본 노래 Playlist] 제목은 모르더라도, 한때 지겹도록 들었던",2:23:30,https://yt3.ggpht.com/a-/AOh14Gjq5YofumMoqhCjq...,하루의 안식처_HAAN,조회수 316만회,5개월 전
5,https://i.ytimg.com/vi/Iy35PgIowcs/hq720.jpg?s...,빠따치기가 취미였던 전설의 마스코트 (사순이 사돌이 1탄),2:22,https://yt3.ggpht.com/a-/AOh14GgPovWeuEAd5Dg2v...,MBC스포츠탐험대,조회수 403만회,7개월 전
6,https://i.ytimg.com/vi/S2Qysqt-7jE/hq720.jpg?s...,[짤툰 오리지널] 금연,3:55,https://yt3.ggpht.com/a-/AOh14Gj07QwC3x3DsbiJT...,짤툰,조회수 268만회,3주 전
